In [ ]:
%pip install keras

In [ ]:
from yfmf import *
import os
from os import walk
from os import path
import tensorflow as tf
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy as scp
from math import erf
import yfinance as yf
import collections
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
from datetime import date
from itertools import cycle, islice
from sklearn.preprocessing import MinMaxScaler
import mplfinance as mpf
import copy
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
f = []
for (dirpath, dirnames, filenames) in walk("data\\"):
    f.extend(filenames)
    break
tickers= [x.replace('.csv', '') for x in filenames]

In [ ]:
'''read data from csv file'''
dfs={}
dates_pdtimestamp={}
date_index_str_split={}

for ticker in tickers:
    try:
        path.exists("data\\"+ticker+".csv")
        dfs[ticker]=pd.read_csv("data\\"+ticker+".csv",index_col=0,parse_dates=True)
        #dfs[ticker].index = dfs[ticker].index.get_level_values(0).astype(int)
        #print(dfs[ticker])
        years=(pd.DatetimeIndex(dfs[ticker].index).year)
        months=(pd.DatetimeIndex(dfs[ticker].index).month)
        days=(pd.DatetimeIndex(dfs[ticker].index).day)
        time=(pd.DatetimeIndex(dfs[ticker].index).time)
 
        y = pd.get_dummies(years)
        y.columns=[str(i) for i in np.unique(years)]
        m = pd.get_dummies(months)
        m.columns=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        d = pd.get_dummies( days)
        d.columns=['1th', '2th', '3th', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th', '21th', '22th', '23th', '24th', '25th', '26th', '27th', '28th', '29th', '30th','31th']
        date=y.append(m).append(d)
        dfs[ticker]=dfs[ticker].append(date)
        dfs[ticker].index=np.arange(0,len(dfs[ticker]),1)
        print(dfs[ticker].head)
    except OSError as e:
         print("Error: %s : %s" % ("data\\"+ticker+".csv", e.strerror))


In [ ]:
'''read data from csv file'''
dfs={}
dates_pdtimestamp={}
date_index_str_split={}

for ticker in tickers:
    try:
        path.exists("data\\"+ticker+".csv")
        dfs[ticker]=pd.read_csv("data\\"+ticker+".csv",index_col=0,parse_dates=True)
        dates_pdtimestamp[ticker]=dfs[ticker].index
        date_index_str=(dfs[ticker].index.astype(str))
        splits=[]
        for string in date_index_str:
            splits.append(string.split('-'))
        Y,D,M=[],[],[]
        for split in splits:
            Y.append(int(split[0]))
            M.append(int(split[1]))
            D.append(int(split[2]))
        dfs[ticker]['Y'],dfs[ticker]['M'],dfs[ticker]['D']=Y,M,D
        dfs[ticker].index=np.arange(0,len(dfs[ticker]),1)
    except OSError as e:
        print("Error: %s : %s" % ("data\\"+ticker+".csv", e.strerror))
dfs['aapl'].head()


In [ ]:
'''plot'''
lim1=0
lim2=-1
means=()
for ticker in tickers:
    data = pd.read_csv("data\\"+ticker+".csv",index_col=0,parse_dates=True)
    plot_stocks(data,lim1,lim2,means,ticker,frmaesize=(5,3),show_nonetrading=False,show_volume=False)

In [ ]:
'''scale values'''
target={}
dfs_sc= copy.deepcopy(dfs)
for ticker in tickers:
    for col in dfs[ticker].keys():
        if col!='D' and col!='Y' and col!='M' and col!='Adj Close':
            dfs_sc[ticker][col] = MinMaxScaler().fit_transform(dfs_sc[ticker][[col]])
    #plot_features(dfs[ticker],dfs_sc[ticker])
    target[ticker]= dfs_sc[ticker].pop('Adj Close')

In [ ]:
'''slice trainsets'''
train_sets={}
dev_sets={}
test_sets={}
train,dev,test,ltrain,ldev,ltest =split_data(dfs_sc['ibm'],target['ibm'],0.9,0.05)
train_shuffle= train.shuffle(ltrain).batch(2**8)
dev_shuffle= dev.shuffle(ldev).batch(16)
test_shuffle= test.shuffle(ltest).batch(16)

In [ ]:
for elements,_ in train_shuffle.take(1):  # only take first element of dataset
    numpy_elements = elements.numpy()
    input_size = (numpy_elements.size)
model = get_compiled_model(input_size)

In [ ]:
fit= model.fit(train_shuffle, epochs=5)

In [ ]:
# list all data in history
print(fit.history.keys())
# summarize history for accuracy
plt.plot(fit.history['accuracy'])
#plt.plot(fit.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(fit.history['loss'])
#plt.plot(fit.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(dev_shuffle)
plt.figure()
plt.plot(predictions)
plt.figure()
plt.plot(target['aapl'][-ldev:])


In [ ]:
# holdDays = 5
# totalDays = dfs_concat_scaled_dataframe.shape[0]
# #mlpr.fit(dfs_concat_scaled_dataframe.index[0:(totalDays-holdDays)].T, dfs_concat_scaled_dataframe['Adj Close'][0:(totalDays-holdDays)])
# mlpr.fit(a, dfs_concat_scaled_dataframe['Adj Close'][0:(totalDays-holdDays)])

In [ ]:
# #Predict the stock price using the model
# pricePredict = mlpr.predict(date_idx)
# #Display the predicted reuslts agains the actual data
# plt.plot(date_idx, adj_close_price)
# plt.plot(date_idx, pricePredict, c='#5aa9ab')
# plt.show()

In [ ]:
# #Number of neurons in the input, output, and hidden layers
# input2 = 1
# output2 = 1
# hidden2 = 50
# #array of layers, 3 hidden and 1 output, along with the tanh activation function 
# layers = [('F', hidden2), ('AF', 'tanh'), ('F', hidden2), ('AF', 'tanh'), ('F', hidden2), ('AF', 'tanh'), ('F', output2)]
# #construct the model and dictate params
# mlpr2 = ANNR([input2], layers, batchSize = 256, maxIter = 10000, tol = 0.01, reg = 1e-4, verbose = True)

In [ ]:
# holdDays = 5
# totalDays = len(date_idx)
# mlpr2.fit(date_idx[0:(totalDays-holdDays)], adj_close_price[0:(totalDays-holdDays)])
# pricePredict2 = mlpr2.predict(date_idx)

In [ ]:
# plt.plot(date_idx, adj_close_price)
# plt.plot(date_idx, pricePredict, c='#5aa9ab')
# plt.plot(date_idx, pricePredict2, c='#8B008B')
# plt.show()

In [ ]:
# pricePredict3 = mlpr2.predict(date_idx[-holdDays:])
# print((pricePredict3-date_idx[-holdDays:])/date_idx[-holdDays:]*100)